# 01 — Data Quality Analysis

**NovaCred Credit Application Governance | DEGO 2606**

**Author:** Jasper Gräfe (Data Engineer)

---

## Objectives

Assess the quality of `raw_credit_applications.json` across four dimensions:

| Dimension | Description |
|-----------|-------------|
| Completeness | Are all required fields present and populated? |
| Consistency | Are values formatted and coded uniformly? |
| Validity | Do values conform to expected ranges and formats? |
| Accuracy | Are values plausible and internally consistent? |

---

## Sections

0. Setup & Data Loading
1. Dataset Overview & Schema Profiling
2. Completeness Analysis
3. Consistency Analysis
4. Validity Analysis
5. Accuracy Analysis
6. Consolidated Quality Report & Clean Export
7. Quality Scores & Governance Notes

---
## Section 0 — Setup & Data Loading

In [ ]:
# To be implemented